# A Real-world Example Using HBV-SASK Hydrological Model

## Installation (from source)

The latest version of the varstool sensitivity and uncertainty package can always be retrieved from its GitHub repo using the following commands:

In [1]:
!git clone https://github.com/vars-tool/vars-tool
!pip install vars-tool/.

Cloning into 'vars-tool'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 1481 (delta 116), reused 153 (delta 67), pack-reused 1270
Receiving objects: 100% (1481/1481), 16.75 MiB | 2.03 MiB/s, done.
Resolving deltas: 100% (908/908), done.
Processing ./vars-tool
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for varstool: filename=varstool-2.1-py2.py3-none-any.whl size=426382 sha256=5bc3a2cedc237f4ceeea13a54d9ec8fe4a6775812863a77130a548e88d06838e
  Stored in directory: /Users/kasrakeshavarz/Library/Caches/pip/wheels/45/b2/f1/bf4d3b73f816f8ab021b7f8e25cbd04136a346155684c778a3
Successfully built varstool
  Attempting uninstall: varstool
    Found existing installation: varstool 2.1
    Uninstalling varstool-2.1:
      Successfully uninstalled varstool-2.1


## HBV-SASK Model - A Real-world Sensitivity Analysis Problem

The HBV-SASK Model is a real-world hydrological model that has been developed within the [**"Watershed Systems Analysis and Modelling Lab"**](https://www.samanrazavi.com) for educational purposes. In this example, we analyze the sensitivity of each parameter of the HBV-SASK model to the simulated streamflow (i.e., output) using the state-of-the-art varstool Python package.

HBV-SASK utilizes daily time-series of precipitation and temperature and produced many outputs, such as streamflow, evapotransipration, etc.

For more information see: Razavi, S., Sheikholeslami, R., Gupta, H. V., & Haghnegahdar, A. (2019). ***VARS-TOOL: A toolbox for comprehensive, efficient, and robust sensitivity and uncertainty analysis.*** Environmental modelling & software, 112, 95-107. doi: https://doi.org/10.1016/j.envsoft.2018.10.005

In [2]:
from varstool import TSVARS, Model

# Loading the "HBV_SASK" model that is included in this package
from varstool.funcs import HBV_SASK

exp_1_model = Model(HBV_SASK)

Let's just see how the model works. The object "Model" is an standard way of wrapping different functions that are going to be used in varstool. Therefore, users must communicate with their function of interest using this simple wrapper class. The arguments of this class can be quickly viewed by the following command:

In [ ]:
Model?

To quickly test the HBV-SASK model with some random numbers as its input, follow the cell below:

In [ ]:
import pandas as pd

params={#name  #value
       'TT'   : -4.00,
       'C0'   : 0.00,
       'ETF'  : 0.00,
       'LP'   : 0.00,
       'FC'   : 50.0,
       'beta' : 1.00,
       'FRAC' : 0.10,
       'K1'   : 0.05,
       'alpha': 1.00,
       'K2'   : 0.00,
       'UBAS' : 1.00,
       'PM'   : 0.50,
}

exp_1_model(pd.Series(params))

HBV-SASK returns many outputs; for the sake of simplicity, here we just use the simulated streamflow as the chosen model response. As HBV-SASK returns time-series of streamflow, the time-series version of the **`VARS`** algorithm is employed. The time-series version of this algorithm is detailed in the following publication:

* Gupta, H. V., & Razavi, S. (2018). Revisiting the basis of sensitivity analysis for dynamical earth system models. Water Resources Research, 54(11), 8692-8717. doi: https://doi.org/10.1029/2018WR022668

In [3]:
# setting up a TSVARS experiment for the HBV-SASK model

exp_1 = TSVARS(num_stars=2,
               parameters={  #lb     #ub
                   'TT'   : [-4.00, 4.00],
                   'C0'   : [0.00 , 10.0],
                   'ETF'  : [0.00 , 1.00],
                   'LP'   : [0.00 , 1.00],
                   'FC'   : [50.0 ,500.0],
                   'beta' : [1.00 , 3.00],
                   'FRAC' : [0.10 , 0.90],
                   'K1'   : [0.05 , 1.00],
                   'alpha': [1.00 , 3.00],
                   'K2'   : [0.00 , 0.05],
                   'UBAS' : [1.00 , 3.00],
                   'PM'   : [0.50 , 2.00],
               },
               delta_h=0.1,
               ivars_scales=(0.1, 0.3, 0.5),
               model=exp_1_model,
               seed=1001,
               sampler='lhs',
               bootstrap_flag=False,
               grouping_flag=False,
               func_eval_method='serial',
               vars_eval_method='serial',
               vars_chunk_size=None,
          )

At any time, the status of the VARS analysis could be viewed by typing the name of the instance that is used to initiate the experiment. Here, it is `exp_1`:

In [ ]:
exp_1

From the status report, it could be viewed that the `star_centres` are fully sampled (via `lhs`) and loaded. Meanwhile, $\Delta$h (`delta_h`) is set to 0.1, `seed` is set to 1001, and 12 parameters are fully set to begin the VARS sensitivity analysis.

Time-series VARS (TSVARS) comes with two computational methods to carry out sensitivity analysis, that are:
1. Serial and
2. Parallel.

It is worth noting that these computational methods are applicable to: **a**) while varstool runs the `Model` and **b**) when varstool computes common `VARS` indices. For the former `func_eval_method` method must be changed and for the latter `vars_eval_method` to either `'serial'` or `'parallel'`.

Furthermore, to save memory usage, the computations can be chunked into several segments to avoid memory leak. In this example, first a simple serial version without chunking is demonstrated. It should be noted that the parallel version are unstable and might result in unexpected errors.

Before carrying out a sensitivity analysis, let's check the loaded `star_centres`:

In [4]:
exp_1.star_centres

array([[0.74408722, 0.96041308, 0.09803003, 0.21526074, 0.01155678,
        0.09789096, 0.9911775 , 0.51015798, 0.85274302, 0.29022856,
        0.6763727 , 0.02056527],
       [0.15311609, 0.13253178, 0.55455094, 0.70552831, 0.73506341,
        0.56364827, 0.17640264, 0.11162101, 0.30676093, 0.98275457,
        0.42678384, 0.80294738]])

It could be seen that 2 star centres for 12 parameters of the HBV-SASK model are successfully produced. Now let's move on to the sensitivity analysis bit:

In [5]:
exp_1.run_online()

In [6]:
exp_1

Star Centres: Loaded
Star Points: Loaded
Parameters: 12 paremeters set
Delta h: 0.1
Model: HBV_SASK
Seed Number: 1001
Bootstrap: Off
Bootstrap Size: N/A
Bootstrap CI: N/A
Grouping: Off
Number of Groups: None
TSVARS Analysis: Done
Function Evaluation Method: serial
TSVARS Evaluation Method: serial
TSVARS Chunk Size: N/A